`pidgin.weave` defines how the input source with be displayed, and always follows `pidgin.tangle`. 

Weaving places `display` as having the same importance as `eval` and `exec` 

lexical analysis i sperforme don non python source.


In [1]:
    import IPython, re, functools, traitlets, typing as t
    with __import__('importnb').Notebook():
        from . import tangle
    if __name__ == '__main__':
        %reload_ext pidgin
        import pidgin

In [6]:
    def weave(result: IPython.core.interactiveshell.ExecutionResult) -> t.Tuple[dict, dict]: return IPython.display.display(result.info.raw_cell)

In [7]:
    def load_ipython_extension(shell: IPython.core.interactiveshell.InteractiveShell):
        shell.display_formatter = WeaveFormatter(**shell.display_formatter._trait_values)
        shell.events.register('post_run_cell', weave)
        for object in formatters_by_name: shell.display_formatter.mimebundle_formatter.for_type_by_name(*object)  

`pidgin.weave` establishes __Markdown__ rules for displaying cell inputs after the `ip.run_cell` has been evaluated.

`pidgin.weave` is a secondary literate program with rules for displaying input encoded in `mistune.markdown`.

In [4]:
    import re, dataclasses, ast, re,traceback, functools, IPython, jinja2, inspect, slugify, collections, html, fnmatch, mistune, inspect, importlib, types, base64, string, traitlets, yaml, io, textwrap
    import nbconvert
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]
    shell = IPython.get_ipython()

    with __import__('importnb').Notebook():
        from . import tangle
    with tangle.Pidgin():
        from . import testing

> The following regular expressions are copied from `mistune`.

`mistune.BlockGrammar and mistune.InlineGrammar` includes doctype and prolog parsing in the html and should be used with `mistune.BlockLexer`.

In [5]:
    class InlineHtmlGrammar:
        """Add svg constructs to inline_html regex."""
        inline_html = re.compile(
                r'^(?:%s|%s|%s)' % (
                    r'<!--[\s\S]*?-->|<!DOCTYPE [\s\S]*?>|<\?[\s\S]*?\?>',
                    r'<(\w+%s)((?:%s)*?)\s*>([\s\S]*?)<\/\1>' % (
                        mistune._valid_end, mistune._valid_attr),
                    r'<\w+%s(?:%s)*?\s*\/?>' % (mistune._valid_end, mistune._valid_attr)))
        
    class InlineGrammar(InlineHtmlGrammar, mistune.InlineGrammar): ...

In [6]:
    error = "background-color: var(--jp-rendermime-error-background);"
    ws = "white-space: pre;"

In [7]:
    def whiten(str: str) -> str: return re.sub('\S{1}', ' ', str)

In [8]:
    class WeaveInlineLexer(mistune.InlineLexer):
        """WeavePidginInlineLexer is responsible for updating modifying the interactive IPython state.
        """
        grammar_class = InlineGrammar

In [9]:
    link = """[{text}]({link} "{title}")"""

In [10]:
    class WeaveRenderer(mistune.Renderer):
        """WeaveRenderer is responsbile for formatting the output of the input code."""
        failure = None
        tmp_code = []
         
        def codespan(self, In: str) -> str:
            all = '\n'.join([*self.tmp_code, In])
            if all.endswith('\\'): 
                self.tmp_code = all.splitlines()
                return super().codespan(In)
            self.tmp_code = []
            Out = ipython_output(F"`{all}`")
            return super().codespan(text) if Out == all else Out
        
        def text(self, text: str) -> str: return text
        
        def doctest(self, In: str) -> str:
            Out = ipython_output(In)
            return super().block_code(In) if In == Out else Out
        
        def link(self, link, title, text):
            ipython_output(link.format(**locals()))
            return super().link(link, title, text)

        def image(self, link, title, text):
            ipython_output('!'+link.format(**locals()))
            return super().image(link, title, text)

add header and footnotes

In [11]:
User statements versus user expressions.

User statements versus user expressions.

In [12]:
    class WeaveMarkdown(mistune.Markdown):
        """WeaveMarkdown combines initializes and applies WeavePidginBlockLexer and WeavePidginInlineLexer.
        """
        def output_doctest(self): return self.renderer.doctest(self.token['text'])
        
        def render(self, text, failure=None):
            self.renderer.failure = failure
            return super().render(text)
        
        def parse(self, text):
            out = self.output(mistune.preprocessing(text))
            keys = self.block.def_footnotes
            self.block.def_footnotes = {}
            self.inline.footnotes = {}
            
            if not self.footnotes: return out

            self.footnotes = sorted(self.footnotes, key=lambda o: keys.get(o['key']), reverse=True)

            body = self.renderer.placeholder()
            while self.footnotes:
                note = self.footnotes.pop()
                body += self.renderer.footnote_item(note['key'], note['text'])

            out += self.renderer.footnotes(body)
            return out

In [13]:
    class BlockLexer(tangle.BlockLexer):
        def __init__(self, rules=None, **kwargs):
            super().__init__(rules, **kwargs)
            self.def_links = IPython.get_ipython().display_formatter.def_links

1. `IPython.core.formatters.DisplayFormatter`
    1. `IPython.core.formatters.IPythonDisplayFormatter`
    2. `IPython.core.formatters.MimeBundleFormatter`
    3. `IPython.core.formatters.DisplayFormatter.formatters`

Lead in with one example of a parser then build out a few more.  The inline parser is proably best.

In [14]:
    def eval_shorthand_ipython(ip, str):
        return eval(ip.input_transformer_manager.transform_cell(textwrap.indent(str, ' '*4)), ip.user_ns, ip.user_global_ns)

In [15]:
    def embed(ip, str):
        type = guess(str) or ''
        if str.startswith('http'): 
            if type.startswith('image') and not type.endswith('svg'):
                return IPython.display.Image(str)
            return IPython.display.IFrame(str, '100%', '500px')


In [16]:
    def graphviz(shell, str): 
        try: import graphviz
        except: return
        if '->' in str and not str.startswith('di'): str = 'di' + str
        return graphviz.Source(str)

In [17]:
    def run_code(shell, raw):
        text, error = raw.strip('`'), None
        body = text.splitlines()[-1]
        with IPython.utils.capture.capture_output(stderr=False) as output: 
            result = shell.run_cell(text)
        print(result)
        error = result.error_in_exec or result.error_before_exec
        return IPython.display.HTML(
    F"""<code style="background: var(--jp-rendermime-error-background);" title="{format_title(str(error))}">{body}</code>"""
        ) if error else IPython.display.HTML(F"<code>{body}</code>")

In [18]:
    def run_block_code(ip, raw):
        return F"<pre>{run_code(ip, raw)}</pre>"

In [19]:
    @dataclasses.dataclass
    class PrettyMarkdown:
        data: str
        def _repr_mimebundle_(self, include=None, exclude=None): return {'text/plain': self.data, 'text/markdown': self.data}, {}

In [20]:
    def run_doctest(ip, text):
        with IPython.utils.capture.capture_output() as out:
            # variables created in the doctest namespace are stored in a specific table.
            runner = testing.run_docstring_examples(text, IPython.get_ipython(), verbose=True)
        runner.failures and  IPython.get_ipython().showtraceback(
            (testing.DocTestException, testing.DocTestException(out.stdout), BaseException().__traceback__))
        return IPython.display.HTML(
    F"""<pre title="{format_title(out.stdout)}"><code style="background: var(--jp-rendermime-error-background);">{text}</code></pre>"""
        ) if runner.failures else IPython.display.Pretty(text)

In [21]:
    def assign_link(ip, raw):
        text, link_title = raw.lstrip('!').lstrip('[').rstrip(')').partition(')[')
        link, title = link_title.split(' ', 1)
        ip.user_ns[link] = ip.user_ns.get(link, link)
        if text.strip(): ip.user_ns[text] = ip.user_ns.get(text, raw)

In [22]:
    def format_title(object): return '&#013;'.join(
        html.escape(nbconvert.filters.strip_ansi(str(object))).splitlines())

In [23]:
    class StringFormatterException(BaseException): ...

In [24]:
    class StringFormatter(IPython.core.formatters.DisplayFormatter):
        """The parent is the displayformatter"""        
        string_formatters = traitlets.Dict(default_value={
            ';[! ]*': eval_shorthand_ipython,
            ',[! ]*': eval_shorthand_ipython,
            'https://*' : embed,
            'http://*' : embed,
            'graph*': graphviz, 
            'digraph*': graphviz,
            '>>> *': run_doctest,
            '`*[!\\]`': run_code,
            '```*[!\\]```': run_code,
            '[*](*)': assign_link,
            '![*](*)': assign_link,})
        
        def for_pattern(self, pattern, callable=None):
            if callable: self.string_formatters[pattern] = callable
            return self.string_formatters.get(pattern, None)
        
        def match_pattern(self, object):
            for pattern in self.string_formatters:
                if fnmatch.fnmatch(object, pattern): return pattern
            
        def format(self, object, include=None, exclude=None):
            value = self.parent.user_ns.get(object, object)                
            if value != object: return self.format(value)

            if isinstance(object, str): 
                pattern = self.match_pattern(object)
                object = self.string_formatters[pattern](self.parent, object) if pattern else object
            return super().format(object, include=include, exclude=exclude)

In [25]:
    class WeaveFormatter(StringFormatter):
        """The parent is the displayformatter"""    
        html = traitlets.Bool(False, help="""Return an annotated and custom html view for the markdown input.""")        
        doctest = traitlets.Bool(True, help="""Evaluate doctests in the source..""")
        inline = traitlets.Bool(True, help="""Evaluate inline code elements..""")
        def_links = traitlets.Dict(default_value={})
        environment = traitlets.Instance(jinja2.Environment)
        
        @traitlets.default('environment')
        def _default_environment(self): return new_environment()
        
        level = traitlets.Int(default_value=0)
        def __enter__(cls): cls.level += 1
        def __exit__(cls, *exc): cls.level -= 1

        def template(self, object, **data): 
            return self.environment.from_string(object).render(
                collections.ChainMap(data, self.parent.user_ns, vars(__import__('builtins'))))
        
        def front_matter(self, str):
            if str.startswith('---'):
                if '\n---' in str:
                    import yaml, io
                    body = str.lstrip('---')
                    data, sep, body = body.partition('---\n')
                    data = yaml.safe_load(io.StringIO(self.environment.from_string(data).render()))
                    if isinstance(data, dict): return body, data
            return str, {}
        
        def format(self, object, include=None, exclude=None):
            if isinstance(object, str): 
                with self:
                    data, metadata = super().format(object, include=include, exclude=exclude)
                    if len(data) > 1 or 'text/plain' not in data:
                        return data, metadata
                    
                    if self.level == 1: 
                        source, metadata = self.front_matter(object)
                        source = self.template(source, **metadata)
                        return {
                            'text/plain': object, 'text/markdown': source, 
                            'text/html': self.template(markdown().render(''.join(source)), **metadata)
                        } if do_publish(object) else {}, metadata
            return super().format(object, include=include, exclude=exclude)


In [26]:
    def markdown() -> mistune.Markdown: 
        return WeaveMarkdown(
        renderer=WeaveRenderer(escape=False, parse_block_html=True, parse_inline_html=False),
        block=BlockLexer, inline=WeaveInlineLexer)

In [27]:
    def do_publish(data: str) -> bool:
        is_display = bool(data.partition('\n')[0].strip()) 
        source = tangle.markdown_to_python(data)
        is_display = is_display & ~bool(only_contains_block_code(data, source))
        try:
            if data.strip() != ast.literal_eval(source.rstrip().rstrip(';')).strip(): is_display = False
        except: ...
        return is_display

In [28]:
    def only_contains_block_code(raw: str, source: str) -> bool: 
        return raw.startswith('%%') or (
            list(map(str.rstrip, textwrap.dedent(raw).splitlines())) == list(map(str.rstrip, textwrap.dedent(source).splitlines())))

In [29]:
    def format_title(object): 
        return '&#013;'.join(html.escape(nbconvert.filters.strip_ansi(str(object))).splitlines())

In [30]:
The weave step is similar to the user_expressions.  Only evaluating expressions was too limiting.

The weave step is similar to the user_expressions.  Only evaluating expressions was too limiting.

In [ ]:
    def unload_ipython_extension(shell):
        shell.display_formatter = IPython.core.formatters.DisplayFormatter(**shell.display_formatter._trait_values)
        shell.events.callbacks['post_run_cell'] = [object for object in shell.events.callbacks['post_run_cell'] if object.__name__ != 'weave']

`run_cell_display` triggers a display on the input if contains anything other than block code.

In [ ]:
## Templating

Putting real data into documents

Templates are common in blogs.

Jinja has a port in javascript nunjucks.


[templating]: https://en.wikipedia.org/wiki/Template_processor
[jekyll]: https://jekyllrb.com/
[fstring]:

In [3]:
    import jinja2, nbconvert, types, collections, IPython
    from mimetypes import guess_type; guess = lambda x: guess_type(x)[0]

In [ ]:
    class TemplateExporter(nbconvert.exporters.TemplateExporter):
        def _create_environment(self):
            environment = super()._create_environment()
            environment.undefined = jinja2.StrictUndefined
            environment.finalize = ipython_output
            return environment
        
        @classmethod
        def new_environment(cls, **kwargs): return cls(**kwargs).environment

    new_environment = TemplateExporter.new_environment

In [ ]:
    def flatten(str): return '&#xA;'.join(str.splitlines())

In [ ]:
    def format_bytes(bytes, object):
        return {
            'text/html': format_images('image/png', {'image/png': bytes}),
            'text/plain': repr(object),}, {}

In [ ]:
    def show_axes(object):
        import matplotlib.backends.backend_agg
        bytes = io.BytesIO()
        matplotlib.backends.backend_agg.FigureCanvasAgg(getattr(object, 'figure', object)).print_png(bytes)
        return format_bytes(bytes.getvalue(), object)

In [ ]:
    def show_sympy_axes(object): 
        s = __import__('io').BytesIO()
        object.save(s)
        return format_bytes(s.getvalue(), object)

In [ ]:
    formatters_by_name = (
        ('matplotlib.figure', 'Axes', show_axes), 
        ('matplotlib.axes._subplots', 'AxesSubplot', show_axes),
        ('sympy.plotting.plot', 'Plot', show_sympy_axes),
    )

In [ ]:
    if __name__ == '__main__':
        with tangle.PidginImporter():
            from . import weave
        unload_ipython_extension(get_ipython())        
        weave.load_ipython_extension(get_ipython())
        load_ipython_extension(get_ipython())

In [4]:
    def ipython_output(object=None, bundle=None):
        if bundle is None:             
            bundle, metadata = IPython.get_ipython().display_formatter.format(object)
        for type in [
            str for str in IPython.get_ipython().display_formatter.active_types if str != 'text/plain'
        ] + ['text/plain']:
            if type == 'text/plain': 
                if object is None: return bundle[type]
                return object
            if type in bundle: 
                object = bundle[type]
                if type.startswith('image') and 'svg' not in type: object = format_images(type, bundle)._repr_html_()
                if type ==  'text/html': object = flatten(object)
                if type == 'image/svg+xml': object = flatten(re.sub('(xmlns(:?\S*)?="\S*?")?', r'', object))
                if type == 'text/latex':
                    if object.startswith('$$') and object.endswith('$$'): object = object[1:-1]
                break
        return object

In [ ]:
    def format_images(type, bundle):
        str = bundle[type]
        import base64, IPython
        
        if isinstance(str, bytes):
            str = base64.b64encode(str).decode('utf-8')
            
        if type in ('image/svg+xml', 'text/html'):  ...
        elif str.startswith('http'): 
            str = F"""<img src="{str}"/>"""
        else: 
            str = F"""<img src="data:{type};base64,{str}"/>"""
            
        return IPython.display.HTML(str)

In [31]:
    if __name__ == '__main__':
        unload_ipython_extension(get_ipython())
        tangle.unload_ipython_extension(get_ipython())
        tangle.load_ipython_extension(get_ipython())
        load_ipython_extension(get_ipython())

In [32]:
    get_ipython().display_formatter.format(IPython.display.Pretty('object'))

TypeError: unhashable type: 'dict'